In [8]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist
import haversine as hs
from scipy.spatial.distance import cdist
from create_dist_matrix import create_dist_matrix, create_dist_mat_2
from get_avail_act import get_avail_act
from get_best_action import get_best_action
from get_or_tools_sol import or_solution
from get_random_traj import get_random_traj
from train import train_model
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
import pandas as pd

data_csv = pd.read_csv("clean_data.csv")
df = pd.DataFrame(data_csv)


In [10]:
df.drop_duplicates(subset=['CourierId','Latitude','Longitude'],keep='first',inplace=True)

df

,CourierId,Latitude,Longitude
0,324293,55.605452,12.583987
3,324293,55.631359,12.656990
8,324293,55.638429,12.634550
12,324293,NaN,NaN
15,324293,55.656790,12.604109
...,...,...,...
254923,0,55.569114,12.618886
254924,0,55.568978,12.619223
254925,0,55.568047,12.618471
254926,0,55.570509,12.620062


In [11]:
df = df.dropna(axis=0)
index_list = [i for i in range(len(df))]
df = df.set_index([index_list])

df

,CourierId,Latitude,Longitude
0,324293,55.605452,12.583987
1,324293,55.631359,12.656990
2,324293,55.638429,12.634550
3,324293,55.656790,12.604109
4,324293,55.657004,12.604765
...,...,...,...
67658,0,55.569114,12.618886
67659,0,55.568978,12.619223
67660,0,55.568047,12.618471
67661,0,55.570509,12.620062


In [12]:
alpha = 0.012
gamma = 0.4
def tsp_solver(dist_mat,alpha=0.2,gamma=0.8):
    #alpha is the learning rate
    #gamma is the discount factor
    n_dest = dist_mat.shape[0]
    # Train RL model
    q = train_model(dist_mat, n_train = 2000, gamma = gamma, alpha = alpha)# Get trained transition matrix

    #print(q)

    # Use model to find optimum trajectory
    state = [0]
    distance_travel = 0.
    posible_actions = get_avail_act(state, n_dest)
    while posible_actions: # until all destinations are visited
        action = get_best_action(state[-1], posible_actions, q)
        distance_travel += dist_mat[state[-1], action]
        state.append(action)
        posible_actions = get_avail_act(state, n_dest)

    #Back to warehouse
    action = 0
    distance_travel += dist_mat[state[-1], action]
    state.append(action)

    # Get Best optimization possible
    #print("\nGoogle Results: ")
    best_dist, google_route = or_solution(dist_mat)

    # Get random tour
    random_dist, random_route = get_random_traj(dist_mat)

    #Out RL results
    traj =' -> '.join([str(b) for b in state])
    #print(f"Best trajectory found with RL: \n {traj}" )
    #print(f"Total distance travelled with this traj: {distance_travel}\n")

    lat_long_array = [(df.Latitude[i],df.Longitude[i]) for i in state]

    slow_pctg = 100*(-1+distance_travel/best_dist)
    random_pctg = 100*(-1+distance_travel/random_dist)
    return slow_pctg, traj, distance_travel, google_route, best_dist, lat_long_array
    #print(f"RL solution is {100*(-1+distance_travel/best_dist)}% slower than google's solution")

dist_list = [0] * len(df)
df['Distance'] = dist_list
new_df = df.loc[df.CourierId == 324293]
new_df = new_df.set_index([[i for i in range(len(new_df))]])
dist_matrix = [[0]*len(new_df) for _ in range(len(new_df))]

for j in new_df.index:
    for k in new_df.index:
        loc1=(new_df.Latitude[j], new_df.Longitude[j])
        loc2=(new_df.Latitude[k], new_df.Longitude[k])
        dist_matrix[j][k] = hs.haversine(loc1,loc2)

dist_matrix = np.array(dist_matrix)

slow_pctg, rl_route, rl_dist, google_route, google_dist,coords_array = tsp_solver(dist_matrix, alpha=alpha, gamma=gamma)
print("The route is : ")
print(rl_route)
print("=======================")
print("the distance travelled is :")
print(rl_dist)
print("=============================")
print("The coordinates of the route :")
print(coords_array)

The route is : 
0 -> 12 -> 11 -> 10 -> 4 -> 3 -> 2 -> 1 -> 5 -> 8 -> 15 -> 7 -> 17 -> 13 -> 16 -> 14 -> 9 -> 6 -> 0
the distance travelled is :
46.62797088260529
The coordinates of the route :
[(55.605452, 12.583987), (55.666729, 12.623844), (55.666922, 12.624307), (55.68444, 12.606241), (55.657004, 12.604765), (55.65679, 12.604109), (55.638429, 12.63455), (55.631359, 12.65699), (55.633779, 12.650921), (55.637848, 12.580652), (55.660381, 12.596661), (55.645927, 12.642146), (55.665391, 12.626504), (55.666784, 12.624088), (55.65963, 12.604466), (55.664992, 12.622379), (55.684063, 12.541127), (55.629509, 12.639366), (55.605452, 12.583987)]


Transforming the array to a Numpy array

In [7]:
coords_array = np.array(coords_array)
coords_array

array([[55.605452, 12.583987],
       [55.657004, 12.604765],
       [55.65679 , 12.604109],
       [55.666729, 12.623844],
       [55.666922, 12.624307],
       [55.68444 , 12.606241],
       [55.684063, 12.541127],
       [55.664992, 12.622379],
       [55.629509, 12.639366],
       [55.665391, 12.626504],
       [55.666784, 12.624088],
       [55.65963 , 12.604466],
       [55.637848, 12.580652],
       [55.633779, 12.650921],
       [55.660381, 12.596661],
       [55.645927, 12.642146],
       [55.638429, 12.63455 ],
       [55.631359, 12.65699 ],
       [55.605452, 12.583987]])